In [1]:
import pandas as pd
from scripts.mock.mock_cleaning import create_demand
import pyomo.environ as pyo

# Data import

In [2]:
nodes = pd.read_pickle("../data/original/nodes.pkl")
channels = pd.read_pickle("../data/original/channels.pkl")

In [3]:
nodes.to_csv("../data/original/nodes.csv")
channels.to_csv("../data/original/channels.csv")

## Modeling

Change the logic of CHANNELS set construction by considering the pair of nodes as the edge identifier.
In this way I also have to deal with:
- [X] multiedges, aka multiple edges between two nodes. I can not consider those for simplicity
- [X] directed relationship. I need to understand if flipping the channels - as done before - can be reasonable. I also need to determine if the flipped pair of nodes is considered a valid index by pyomo.

In [4]:
## Note that the following are arbitrary policies, that can be changed as needed.
## Deal with multi-edges (aka multiple channels between two peers):
## - average rate fee
## - average base fee,
## - average capacity
## - keep one of the two channel ids
aggregation_dict = {
    "channel_id": "first",
    "rate_fee": "mean",
    "base_fee": "mean",
    "capacity": "sum"
}
channels.reset_index(inplace=True)
channels = channels.groupby(["node1_pub", "node2_pub"]).agg(aggregation_dict)
channels.reset_index(inplace=True)
channels.set_index("channel_id", inplace=True)

In [5]:
## To delete from [outgoing] and [incoming] lists the multi-edge
for i in nodes.index:
    chan = []
    for l in nodes.loc[i, "outgoing_channels"]:
        if l in channels.index:
            chan.append(l)
    nodes.at[i, "outgoing_channels"] = chan

for i in nodes.index:
    chan = []
    for l in nodes.loc[i, "incoming_channels"]:
        if l in channels.index:
            chan.append(l)
    nodes.at[i, "incoming_channels"] = chan

In [6]:
model = pyo.ConcreteModel(name="Min cost flow problem")
model.NODES = pyo.Set(initialize=nodes.index)
model.CHANNELS = pyo.Set(initialize=[(channels.loc[i, "node1_pub"], channels.loc[i, "node2_pub"]) for i in channels.index])

In [7]:
nodes = create_demand(nodes, 1000)

Transaction of 1000 sats.
Sender: gocharlie-lightning-node-1
Receiver: the_holy_cheese_grater.


In [8]:
model.x = pyo.Var(model.CHANNELS, domain=pyo.Binary)
model.a = pyo.Var(model.CHANNELS, domain=pyo.NonNegativeReals, bounds=(0, max(nodes["demand"])))

In [9]:
channels.reset_index(inplace=True)
channels.set_index(["node1_pub", "node2_pub"], inplace=True)
channels.sort_index(inplace=True)

In [10]:
def objective_function(model: pyo.ConcreteModel):
    return sum(model.a[i] * channels.loc[i, "rate_fee"] for i in model.CHANNELS) + sum(model.x[i] * channels.loc[i, "base_fee"] for i in model.CHANNELS)

model.totalCost = pyo.Objective(rule=objective_function(model), sense=pyo.minimize)

### Constraints

#### Capacity constraint

$$amount_{i,j} \le capacity_{i,j} \times x_{i,j} \text{ } \forall (i,j) \in E$$


In [11]:
def capacity_constraint(model: pyo.ConcreteModel, a, b):
    return model.a[(a, b)] <=  channels.loc[(a, b), "capacity"] * (1 if channels.loc[(a,b), "base_fee"]<1 else model.x[(a, b)])

model.CapacityConstraint = pyo.Constraint(model.CHANNELS, rule=capacity_constraint, name="Capacity constraint")

#### Flow balance constraint

$$\sum_{(s,i) \in E} amount_{si} - \sum_{(i,t) \in E} amount_{it} = b_i \text{ } \forall i \in V$$


In [12]:
channels.reset_index(inplace=True)
channels.set_index("channel_id", inplace=True)

def flow_balance_constraint(model: pyo.ConcreteModel, n: str):
    return sum(model.a[(channels.loc[a, "node1_pub"], channels.loc[a, "node2_pub"])] for a in nodes.loc[n, 'incoming_channels']) - sum(model.a[(channels.loc[a, "node1_pub"], channels.loc[a, "node2_pub"])] for a in nodes.loc[n, 'outgoing_channels']) == nodes.loc[n, "demand"]

model.FlowBalanceConstraint = pyo.Constraint(model.NODES, rule=flow_balance_constraint, name="Flow balance constrain")

channels.reset_index(inplace=True)
channels.set_index(["node1_pub", "node2_pub"], inplace=True)
channels.sort_index(inplace=True) 

## Solving the model

In [13]:
opt = pyo.SolverFactory('cbc')
results = opt.solve(model, tee=True)

if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
    print('\nOptimal solution found')
elif results.solver.termination_condition == pyo.TerminationCondition.feasible:
    print('\nFeasible but not proven optimal solution found')
elif results.solver.termination_condition == pyo.TerminationCondition.infeasible:
    raise Exception("The model is infeasible")
else:
    print('\nSolver Status: ',  results.solver.status)
    raise Exception(results.solver.status)

print('\nObject function value = ', model.Objective())


Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: May  9 2022 

command line - /usr/bin/cbc -printingOptions all -import /tmp/tmp3fxz9iki.pyomo.lp -stat=1 -solve -solu /tmp/tmp3fxz9iki.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 27648 (-25216) rows, 68669 (-7243) columns and 132589 (-32237) elements
Statistics for presolved model
Original problem has 28914 integers (28914 of which binary)
Presolved problem has 24206 integers (24206 of which binary)
==== 3511 zero objective 2665 different
==== absolute objective values 2665 different
==== for integers 0 zero objective 389 different
==== for integers absolute objective values 389 different
===== end objective counts


Problem has 27648 rows, 68669 columns (65158 with objective) and 132589 elements
There are 25084 singletons with objective 
Column breakdown:
0 of type 0.0->inf, 44463 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of t

In [14]:
from decimal import Decimal
DF = pd.DataFrame()
c = 0
for index, value in model.a.extract_values().items():
    DF.loc[c, "amount"] = Decimal(value)
    DF.loc[c, "source"] = index[0]
    DF.loc[c, "destination"] = index[1]
    c += 1

In [18]:
DF[DF["amount"]>0]

,amount,source,destination
6482,1000,0255f20d53b9a53c0d22268217894ccb90b11b86f95609...,029efe15ef5f0fcc2fdd6b910405e78056b28c9b64e1fe...
9189,500,0270a26e0772d0cde73d870cbadf632c5377f83ed02d97...,03011e480a671ac71dbc3fc3e8fe0db32bb9a10cc4d824...
10780,1000,02881003525adb05fef7f73eacc8f31903791ced708a4f...,035e4ff418fc8b5554c5d9eea66396c227bd429a3251c8...
10874,500,02896feeb3e4146691f8110a0c39cb785064ce40a4e320...,02bd3e6dfab3347a9b058777ea075ded4b77605eece977...
16868,500,02cfdc6b60e5931d174a342b20b50d6a2a17c6e4ef8e07...,0270a26e0772d0cde73d870cbadf632c5377f83ed02d97...
17157,500,02cfdc6b60e5931d174a342b20b50d6a2a17c6e4ef8e07...,03a93b87bf9f052b8e862d51ebbac4ce5e97b5f4137563...
30288,1000,035e4ff418fc8b5554c5d9eea66396c227bd429a3251c8...,0255f20d53b9a53c0d22268217894ccb90b11b86f95609...
31428,500,03641a88d80a2a85bbecd770577aca9b5495616e9fef63...,03011e480a671ac71dbc3fc3e8fe0db32bb9a10cc4d824...
31447,500,03641a88d80a2a85bbecd770577aca9b5495616e9fef63...,03a93b87bf9f052b8e862d51ebbac4ce5e97b5f4137563...
31760,1000,0364913d18a19c671bb36dd04d6ad5be0fe8f2894314c3...,02cca6c5c966fcf61d121e3a70e03a1cd9eeeea024b26e...


In [16]:
nodes.reset_index(inplace=True)
DF_nodes = nodes[nodes["pub_key"].isin(DF.loc[DF["amount"] > 0, "source"].tolist()) & nodes["pub_key"].isin(DF.loc[DF["amount"] > 0, "source"].tolist())]
nodes.set_index("pub_key", inplace=True)


In [19]:
DF_nodes

,pub_key,alias,addresses,outgoing_channels,incoming_channels,demand
953,0255f20d53b9a53c0d22268217894ccb90b11b86f95609...,Wayfinder🦆,1,"[892269079203676166, 892269079203676167, 89226...","[INV892269079203676166, INV892269079203676167,...",0
1215,0270a26e0772d0cde73d870cbadf632c5377f83ed02d97...,Veritas Immortalis,3,"[880957303602937857, 880957303602937858, 88854...","[INV880957303602937857, INV880957303602937858,...",0
1467,02881003525adb05fef7f73eacc8f31903791ced708a4f...,gocharlie-lightning-node-1,1,"[913568818378309632, 913595206853197825, 91363...","[INV913568818378309632, INV913595206853197825,...",-1000
1478,02896feeb3e4146691f8110a0c39cb785064ce40a4e320...,Bank of Japan,1,"[921231314997280769, 921639233795653633, 92211...","[INV921231314997280769, INV921639233795653633,...",0
2305,02cfdc6b60e5931d174a342b20b50d6a2a17c6e4ef8e07...,Voltage-C2,3,"[877709346199437312, 877908357781782532, 87812...","[INV877709346199437312, INV877908357781782532,...",0
3950,035e4ff418fc8b5554c5d9eea66396c227bd429a3251c8...,WalletOfSatoshi.com,2,"[770826920361984001, 877632380469575681, 87768...","[INV770826920361984001, INV877632380469575681,...",0
4038,03641a88d80a2a85bbecd770577aca9b5495616e9fef63...,Nodelou🐈,3,"[878189832724283393, 879275050761977857, 88256...","[INV878189832724283393, INV879275050761977857,...",0
4044,0364913d18a19c671bb36dd04d6ad5be0fe8f2894314c3...,LQwD-Canada,3,"[886592300790185991, 886592300790185996, 88659...","[INV886592300790185991, INV886592300790185996,...",0
4754,03a09f56bba3d2c200cc55eda2f1f069564a97c1fb7434...,Einundzwanzig,3,"[883498275136798720, 883719276806995969, 88385...","[INV883498275136798720, INV883719276806995969,...",0
4850,03a93b87bf9f052b8e862d51ebbac4ce5e97b5f4137563...,cyberdyne.sh,3,"[878739588584177665, 879336623436333057, 88035...","[INV878739588584177665, INV879336623436333057,...",0
